# Communications point-à-point

* Elles sont _locales_, c'est-à-dire que seuls **deux
  processus** sont impliqués : ils sont les seuls à
  appeler réciproquement des fonctions de communication,
  donc ils sont les seuls à participer à cet échange
    * Note : les appels aux fonctions d'envoi et de réception
      viennent toujours par paires sur des processus différents
* Les fonctions d’envoi et de réception peuvent être bloquantes ou non

## `MPI_Send()` (bloquant)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#section.3.2) :

```C
int MPI_Send(void *envoi, int compte, MPI_Datatype type,
             int dest, int etiquette, MPI_Comm comm);
```

* `envoi` : adresse en mémoire du premier élément à envoyer
* `compte` : nombre d’éléments de type `type`
* `type` : type MPI tel que `MPI_INT`, `MPI_DOUBLE`, etc.
* `dest` : rang du processus recevant
* `etiquette` : nombre entier identifiant le type de transfert

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.send) :

```Python
comm.send(objet, dest=dest, tag=etiquette)
```

* `objet` : n'importe quel objet sérialisable via
  [`pickle`](https://docs.python.org/3/library/pickle.html#module-pickle)

## `MPI_Recv()` (bloquant)

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#subsection.3.2.4) :

```C
int MPI_Recv(void *recept, int compte, MPI_Datatype type,
             int source, int etiquette, MPI_Comm comm,
             MPI_Status *etat);
```

* `recept` : adresse en mémoire du premier élément à recevoir
* `compte` : nombre maximal d'éléments de type `type`
* `source` : rang du processus transmettant,
  ça peut être `MPI_ANY_SOURCE`
* `etiquette` : ça peut être `MPI_ANY_TAG`
* `etat` : informations sur le transfert
    * `.count` : nombre d’éléments reçus de type `type`
    * `.MPI_SOURCE` : rang de la source des données
    * `.MPI_TAG` : étiquette du transfert

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.recv) :

```Python
objet = comm.recv(source=source, tag=etiquette, status=etat)
```

* `objet` : une variable ou une partie d'un objet modifiable,
  pour recevoir l'objet désérialisé
* `source` : `MPI.ANY_SOURCE` (valeur par défaut) ou un rang précis
* `etiquette` : `MPI.ANY_TAG` (valeur par défaut) ou un nombre précis
* `etat` : `None` (valeur par défaut) ou objet de type `MPI.Status`
    * `.count` : nombre d’octets reçus
    * `.source` : rang de la source des données
    * `.tag` : étiquette du transfert

### Exemple - `MPI_Send` et `MPI_Recv`

Chaque processus a ses propres variables `a` et `b`.
Le processus 2 envoie la valeur de son `a` vers
le processus 0 qui reçoit cette valeur via son `b`.

![Figure - Communication point à point](images/mpi_point2point.svg)

En **C** :

```C
if (proc == 2) {
    MPI_Send(&a, 1, MPI_INT, 0, 746, MPI_COMM_WORLD);
}
else if (proc == 0) {
    MPI_Recv(&b, 1, MPI_INT, 2, 746, MPI_COMM_WORLD, &etat);
}
```

En **Python** :

```Python
if proc == 2:
    MPI.COMM_WORLD.send(a, dest=0, tag=746)
elif proc == 0:
    b = MPI.COMM_WORLD.recv(source=2, tag=746)
```

### Exercice #2 - Envoi d'une matrice

Votre objectif : envoyer une matrice 4x4 du processus 0 au processus 1 :

1. Dans le répertoire `exercices`, éditez le fichier `send_matrix.c`
   (ou `send_matrix.py`) pour programmer le transfert de la matrice
1. Compilez le code et lancez-le avec deux (2) processus

## Éviter les situations d'interblocage

Soit le code C suivant :

```C
if (proc == 0) {
    MPI_Ssend(&a, 1, MPI_INT, 2, 10, MPI_COMM_WORLD);
    MPI_Recv( &b, 1, MPI_INT, 2, 11, MPI_COMM_WORLD, &etat);
}
else if (proc == 2) {
    MPI_Ssend(&b, 1, MPI_INT, 0, 11, MPI_COMM_WORLD);
    MPI_Recv( &a, 1, MPI_INT, 0, 10, MPI_COMM_WORLD, &etat);
}
```

* [`MPI_Ssend()`](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#page=70)
  (ou [`comm.Send()`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.Send)
  en Python) est une version sans mémoire
  tampon, ce qui la rend toujours bloquante
* Dans le cas ci-haut, les deux processus attendent que l’autre
  fasse appel à `MPI_Recv`. Bref, ce code est **erroné**!
* Avec `MPI_Send` (ou `comm.send()` en Python), le code
  resterait à risque ; la quantité de mémoire tampon étant
  limitée, le code bloquera lors de l’échange de gros messages

### Solution 1

On change l'ordre des appels à `MPI_Ssend`
et `MPI_Recv` sur un des deux processus :

```C
if (proc == 0) {
    MPI_Ssend(&a, 1, MPI_INT, 2, 10, MPI_COMM_WORLD);
    MPI_Recv( &b, 1, MPI_INT, 2, 11, MPI_COMM_WORLD, &etat);
}
else if (proc == 2) {
    MPI_Recv( &a, 1, MPI_INT, 0, 10, MPI_COMM_WORLD, &etat);
    MPI_Ssend(&b, 1, MPI_INT, 0, 11, MPI_COMM_WORLD);
}
```

On peut généraliser la technique à plus de processus :

* Les processus pairs commencent par envoyer
* Les processus impairs commencent par recevoir

### Exercice #3 - Échange de vecteurs

L'objectif : échanger un petit vecteur de données.

1. Dans le répertoire `exercices`, éditez le fichier `exchange.c`
   (ou `exchange.py`) pour programmer l’échange de données
1. Compilez le code et lancez-le avec deux (2) processus

### Solution 2

On utilise des communications non bloquantes pour démarrer
les transferts. Ainsi, même si l'envoi n'est pas terminé,
on peut commencer la réception tout en évitant l'interblocage.

Exemple en **C** :

```C
if (proc == 0) {
    MPI_Isend(&a, 1, MPI_INT, 2, 10, MPI_COMM_WORLD, &requete);
    MPI_Recv( &b, 1, MPI_INT, 2, 11, MPI_COMM_WORLD, &etat);
}
else if (proc == 2) {
    MPI_Isend(&b, 1, MPI_INT, 0, 11, MPI_COMM_WORLD, &requete);
    MPI_Recv( &a, 1, MPI_INT, 0, 10, MPI_COMM_WORLD, &etat);
}

MPI_Wait(&requete, &etat);
```

Exemple en **Python** :

```Python
if proc == 0:
    requete = comm.isend(a, dest=2, tag=10)
    b = comm.recv(source=2, tag=11)
elif proc == 2:
    requete = comm.isend(b, dest=0, tag=11)
    a = comm.recv(source=0, tag=10)

requete.wait()
```

## Communications non bloquantes

[En **C**](https://www.mpi-forum.org/docs/mpi-4.1/mpi41-report.pdf#subsection.3.7.2) :

```C
int MPI_Isend(void *envoi, int compte, MPI_Datatype type,
              int dest, int etiquette, MPI_Comm comm,
              MPI_Request *requete);

int MPI_Irecv(void *recept, int compte, MPI_Datatype type,
              int source, int etiquette, MPI_Comm comm,
              MPI_Request *requete);
```

* Il n'est pas nécessaire que l'envoi et la réception
  soient tous les deux bloquants ou non bloquants
  (toutes les combinaisons sont permises)
* Après l'appel à `MPI_Isend` ou `MPI_Irecv`,
  on doit utiliser l'argument `requete` pour
  s'assurer que la communication est complétée

```C
int MPI_Wait(MPI_Request *requete, MPI_Status *etat);
```

* `MPI_Wait` est bloquante.
  Elle retourne quand la communication liée à `requete` est terminée
* Quand la communication a terminé, on peut
  réutiliser les vecteurs transmis ou reçus

[En **Python**](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html) :

```Python
requete = comm.isend(envoi, dest, tag=0)
requete.wait()

requete = comm.irecv(source=ANY_SOURCE, tag=ANY_TAG)
recept = requete.wait()
```

* Lors de la réception, c'est
  [`requete.wait()`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Request.html#mpi4py.MPI.Request.wait)
  qui retourne l'objet reçu